In [0]:
%pip install databricks-feature-engineering==0.6 databricks-sdk==0.32.0
%restart_python

In [0]:

import os
import time

import mlflow
import pandas as pd
import requests
from databricks import feature_engineering
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession

from hotel_reservation.config import ProjectConfig
from hotel_reservation.serving.feature_serving import FeatureServing

from hotel_reservation.utils import configure_logging

logger = configure_logging("Hotel Reservations Deploy Model")

# Load project config
config = ProjectConfig.from_yaml(config_path="../project_config.yml")

# COMMAND ----------
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

fe = feature_engineering.FeatureEngineeringClient()
mlflow.set_registry_uri("databricks-uc")

# get environment variables
os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

catalog_name = config.catalog_name
schema_name = config.schema_name
feature_table_name = f"{catalog_name}.{schema_name}.hotel_reservation_features"
feature_spec_name = f"{catalog_name}.{schema_name}.return_predictions"
endpoint_name = "hotel-reservations-feature-serving"


In [0]:
# train_set = spark.table(f"{catalog_name}.{schema_name}.train_dataset").toPandas()
# test_set = spark.table(f"{catalog_name}.{schema_name}.test_dataset").toPandas()
# df = pd.concat([train_set, test_set])

# model = mlflow.pyfunc.load_model(f"models:/{catalog_name}.{schema_name}.hotel_reservation_model_fe@latest-model")

# preds_df = df[["Booking_ID", "no_of_adults", "no_of_children", "avg_price_per_room"]]

# input_features = ['no_of_weekend_nights',
#  'no_of_week_nights',
#  'required_car_parking_space',
#  'lead_time',
#  'repeated_guest',
#  'no_of_previous_cancellations',
#  'no_of_previous_bookings_not_canceled',
#  'no_of_special_requests',
#  'Booking_ID',
#  'market_segment_type_Complementary',
#  'market_segment_type_Corporate',
#  'market_segment_type_Offline',
#  'market_segment_type_Online',
#  'type_of_meal_plan_Meal_Plan_2',
#  'room_type_reserved_Room_Type_4',
#  'room_type_reserved_Room_Type_6',
#  'room_type_reserved_Room_Type_7',
#  'quarter_Q2',
#  'quarter_Q3',
#  'quarter_Q4',
#  'no_of_adults',
#  'no_of_children',
#  'avg_price_per_room']

# preds_df["Predicted_status"] = model.predict(df[input_features])
# preds_df = spark.createDataFrame(preds_df)

# fe.create_table(
#     name=feature_table_name, primary_keys=["Booking_ID"], df=preds_df, description="Hotel reservation predictions feature table"
# )

In [0]:
spark.sql(f"""
          ALTER TABLE {feature_table_name}
          SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
        """)

# Initialize feature store manager
feature_serving = FeatureServing(
    feature_table_name=feature_table_name, feature_spec_name=feature_spec_name, endpoint_name=endpoint_name
)

# Create online table
feature_serving.create_online_table()

# Create feature spec
feature_serving.create_feature_spec()

In [0]:
# Deploy feature serving endpoint
feature_serving.deploy_or_update_serving_endpoint()

In [0]:

os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

serving_endpoint = f"https://{os.environ['DBR_HOST']}/serving-endpoints/{endpoint_name}/invocations"
# response = requests.post(
#     f"{serving_endpoint}",
#     headers={"Authorization": f"Bearer {os.environ['DBR_TOKEN']}"},
#     json={"dataframe_records": [{"Booking_ID": "INN00015"}]},
# )

In [0]:
# another way to call the endpoint
start_time = time.time()
response = requests.post(
    f"{serving_endpoint}",
    headers={"Authorization": f"Bearer {os.environ['DBR_TOKEN']}"},
    json={"dataframe_split": {"columns": ["Booking_ID"], "data": [["INN00015"]]}},
)
end_time = time.time()
execution_time = end_time - start_time


logger.info("Response status:", response.status_code)
logger.info("Reponse text:", response.text)
logger.info("Execution time:", execution_time, "seconds")
